In [1]:
import bw2io as bw2io
import bw2data as bw2data
import bw2calc as bw2calc
import pandas as pd
import plotly.subplots as sp
import plotly.graph_objects as go
from plotly.offline import plot
from collections import defaultdict

# from premise import *

In [2]:
bw2io.__version__

(0, 8, 12)

In [3]:
bw2data.__version__

(3, 6, 6)

In [2]:
bw2data.projects.set_current("poppies")

In [5]:
if "ecoinvent 3.9.1 cutoff" not in bw2data.databases:
    ei391cutoff = bw2data.SingleOutputEcospold2Importer(
        r"H:\Data\ecoinvent\ecoinvent 3.9.1_cutoff_ecoSpold02\datasets",
        "ecoinvent 3.9.1 cutoff",
    )
    ei391cutoff.apply_strategies()
    ei391cutoff.statistics()

In [ ]:
len(bw2data.Database("ecoinvent 3.9.1 cutoff"))

21238

# Poppies foreground import

In [ ]:
# Update the file path to where Poppies LCI.xlsx is saved on your local drive
fp = bw2io.ExcelImporter(r"C:\Users\zhang_x\Documents\Poppies\Poppies LCI.xlsx")
fp.apply_strategies()
fp.match_database(
    "ecoinvent 3.9.1 cutoff", fields=("name", "unit", "location", "reference product")
)
fp.match_database(fields=("name", "unit", "location"))
fp.statistics()

Extracted 1 worksheets in 0.19 seconds
Applying strategy: csv_restore_tuples
Applying strategy: csv_restore_booleans
Applying strategy: csv_numerize
Applying strategy: csv_drop_unknown
Applying strategy: csv_add_missing_exchanges_section
Applying strategy: normalize_units
Applying strategy: normalize_biosphere_categories
Applying strategy: normalize_biosphere_names
Applying strategy: strip_biosphere_exc_locations
Applying strategy: set_code_by_activity_hash
Applying strategy: link_iterable_by_fields
Applying strategy: assign_only_product_as_production
Applying strategy: link_technosphere_by_activity_hash
Applying strategy: drop_falsey_uncertainty_fields_but_keep_zeros
Applying strategy: convert_uncertainty_types_to_integers
Applying strategy: convert_activity_parameters_to_list
Applied 16 strategies in 19.15 seconds
Applying strategy: link_iterable_by_fields
Applying strategy: link_iterable_by_fields
17 datasets
93 exchanges
0 unlinked exchanges
  


(17, 93, 0)

In [ ]:
import numpy as np

np.__version__

'1.24.0'

In [8]:
for ds in fp.data:
    if ds.get("comment") is None:
        ds["comment"] = ""

In [43]:
for ds in fp.data:
    for exc in ds["exchanges"]:
        if type(exc["amount"]) == "int":
            exc["amount"] = float(exc["amount"])

In [10]:
fp.write_database()

Writing activities to SQLite3 database:
0% [#################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Title: Writing activities to SQLite3 database:
  Started: 07/09/2024 13:40:19
  Finished: 07/09/2024 13:40:19
  Total time elapsed: 00:00:00
  CPU %: 0.00
  Memory %: 0.09
Created database: Poppies


In [11]:
len(bw2data.Database("Poppies"))

17

# LCIA Methods definition

In [12]:
chosen_methods = [
    ("IPCC 2021", "climate change", "global warming potential (GWP100)"),
    ("EF v3.0 EN15804", "climate change", "global warming potential (GWP100)"),
    (
        "Cumulative Energy Demand (CED)",
        "energy resources: non-renewable",
        "energy content (HHV)",
    ),
    ("EF v3.1", "acidification", "accumulated exceedance (AE)"),
    ("EF v3.1", "climate change", "global warming potential (GWP100)"),
    # ('EF v3.1', 'climate change: biogenic', 'global warming potential (GWP100)'),
    # ('EF v3.1', 'climate change: fossil', 'global warming potential (GWP100)'),
    # ('EF v3.1','climate change: land use and land use change','global warming potential (GWP100)'),
    (
        "EF v3.1",
        "ecotoxicity: freshwater",
        "comparative toxic unit for ecosystems (CTUe)",
    ),
    # ('EF v3.1','ecotoxicity: freshwater, inorganics','comparative toxic unit for ecosystems (CTUe)'),
    # ('EF v3.1','ecotoxicity: freshwater, organics','comparative toxic unit for ecosystems (CTUe)'),
    (
        "EF v3.1",
        "energy resources: non-renewable",
        "abiotic depletion potential (ADP): fossil fuels",
    ),
    (
        "EF v3.1",
        "eutrophication: freshwater",
        "fraction of nutrients reaching freshwater end compartment (P)",
    ),
    (
        "EF v3.1",
        "eutrophication: marine",
        "fraction of nutrients reaching marine end compartment (N)",
    ),
    ("EF v3.1", "eutrophication: terrestrial", "accumulated exceedance (AE)"),
    (
        "EF v3.1",
        "human toxicity: carcinogenic",
        "comparative toxic unit for human (CTUh)",
    ),
    # ('EF v3.1','human toxicity: carcinogenic, inorganics','comparative toxic unit for human (CTUh)'),
    # ('EF v3.1','human toxicity: carcinogenic, organics','comparative toxic unit for human (CTUh)'),
    (
        "EF v3.1",
        "human toxicity: non-carcinogenic",
        "comparative toxic unit for human (CTUh)",
    ),
    # ('EF v3.1','human toxicity: non-carcinogenic, inorganics','comparative toxic unit for human (CTUh)'),
    # ('EF v3.1','human toxicity: non-carcinogenic, organics','comparative toxic unit for human (CTUh)'),
    (
        "EF v3.1",
        "ionising radiation: human health",
        "human exposure efficiency relative to u235",
    ),
    ("EF v3.1", "land use", "soil quality index"),
    (
        "EF v3.1",
        "material resources: metals/minerals",
        "abiotic depletion potential (ADP): elements (ultimate reserves)",
    ),
    ("EF v3.1", "ozone depletion", "ozone depletion potential (ODP)"),
    ("EF v3.1", "particulate matter formation", "impact on human health"),
    (
        "EF v3.1",
        "photochemical oxidant formation: human health",
        "tropospheric ozone concentration increase",
    ),
    (
        "EF v3.1",
        "water use",
        "user deprivation potential (deprivation-weighted water consumption)",
    ),
]
impact_names = [
    "IPCC 2021",
    "IPCC 2021 w bio C",
    "Cumulated energy demand, non-renewable",
    "Acidification",
    "Climate change",
    "Ecotoxicity, freshwater",
    "Resource depletion, fossils",
    "Eutrophication, freshwater",
    "Eutrophication, marine",
    "Eutrophication, terrestrial",
    "Human toxicity, cancer",
    "Human toxicity, non-cancer",
    "Ionising radiation",
    "Land use",
    "Resource depletion, minerals and metals",
    "Ozone depletion",
    "EF-particulate matter",
    "Photochemical ozone formation",
    "Water use",
]
lcia_methods = dict(zip(impact_names, chosen_methods))

# Test Calcuation

In [6]:
ds = [ds for ds in bw2data.Database("Poppies") if ds["name"] == "Poppies district"][0]
ds

'Poppies district' (unit, NL, None)

In [7]:
ipcc = ("IPCC 2021", "climate change", "global warming potential (GWP100)")
lca = bw2calc.LCA({ds: 1}, ipcc)
lca.lci()
lca.lcia()
lca.score / 5097.7 / 50  # 5097.7 of gross floor area; 50 years of lifetime for the buildings

11.973556663692024

In [24]:
elec_nl = [
    ds
    for ds in bw2data.Database("ecoinvent 3.9.1 cutoff")
    if ds["name"] == "market for electricity, low voltage" and ds["location"] == "NL"
]


elec_nl_rsd = [
    ds
    for ds in bw2data.Database("ecoinvent 3.9.1 cutoff")
    if ds["name"] == "electricity, low voltage, residual mix" and ds["location"] == "NL"
]

In [25]:
ipcc = ("IPCC 2021", "climate change", "global warming potential (GWP100)")
lca = bw2calc.LCA({elec_nl[0]: 1}, ipcc)
lca.lci()
lca.lcia()
lca.score

0.4913167023027979

In [27]:
ipcc = ("IPCC 2021", "climate change", "global warming potential (GWP100)")
lca = bw2calc.LCA({elec_nl_rsd[0]: 1}, ipcc)
lca.lci()
lca.lcia()
lca.score

0.4827707856843875

# Calculation Functions

In [231]:
def contribution_LCA_to_df(datasets, methods, amount=1, names=None):
    """
    Performs a Life Cycle Assessment (LCA) on the given datasets and returns the results as a DataFrame.

    Parameters:
    datasets (list): A list of datasets to perform the LCA on
    methods (list): A list of methods to use in the LCA.
    amount (float, optional): The amount to use in the LCA. Defaults to 1
    names (list, optional): A list of names to use in the LCA. Defaults to ['name'].

    Returns:
    pd.DataFrame: A DataFrame containing the results of the LCA.
    """

    if names is None:

        names = ["name"]

    results = defaultdict(dict)

    codes = defaultdict(dict)

    index_dict = {ds["code"]: [ds[i] for i in names][0] for ds in datasets}

    for impact_name, method in lcia_methods.items():

        lca.switch_method(method)

        cf_dict = dict(bw2data.Method(method).load())

        for dataset in datasets:

            dataset_code = dataset["code"]

            for exc in dataset.technosphere():

                if exc["amount"] == 0:
                    continue

                key = (dataset_code, exc.input["name"])

                if exc["input"] in codes[impact_name]:

                    results[impact_name][key] = amount * codes[impact_name][
                        exc["input"]
                    ] * exc["amount"] + results[impact_name].get(key, 0)

                else:

                    lca.redo_lcia({exc.input: exc["amount"]})

                    results[impact_name][key] = lca.score * amount + results[
                        impact_name
                    ].get(key, 0)

                    codes[impact_name][exc["input"]] = lca.score / exc["amount"]

            for exc in dataset.biosphere():

                if exc.input in cf_dict:  # Not all flows are characterized

                    key = (dataset_code, exc.input["name"])

                    results[impact_name][key] = amount * exc["amount"] * cf_dict[
                        exc.input
                    ] + results[impact_name].get(key, 0)

    return pd.DataFrame(results).sort_index(axis=1).rename(index=index_dict)

In [86]:
def group_into_others(df, pct_lim):
    """
    Groups all processes that contribute less than a specified amount to the results for each LCIA method into 'other'.

    Parameters:
    df (pd.DataFrame): The DataFrame to process.
    pct_lim (float): The percentage limit for grouping processes into 'other'.

    Returns:
    pd.DataFrame: The processed DataFrame.
    """

    pct = df.apply(lambda row: row / row.sum(), axis=1)

    # Check if any part of the dataframe is less than pct_lim
    if (pct < pct_lim).any().any():

        # Calculate the normalized values
        normalized_df = df.divide(df.sum(axis=0))

        # Identify the rows to group into 'other'
        other_rows = normalized_df[normalized_df < pct_lim].dropna()

        # Calculate the sum of the 'other' rows
        others_sum = df.loc[other_rows.index].sum()

        # Create a new DataFrame for the 'other' rows
        others_df = pd.Series(
            others_sum,
            index=pd.MultiIndex.from_product(
                [normalized_df.index.get_level_values(0).unique(), ["Others"]]
            ),
        )
        others_df.name = impact_names[0]

        # Drop the 'other' rows from the original DataFrame and append the 'other' DataFrame
        result_df = pd.concat(
            [df.drop(other_rows.index, axis=0), pd.DataFrame(others_df)], axis=0
        )

        return result_df

    else:

        return df

In [193]:
def iterative_contribution_LCA_to_df(
    datasets, methods, pct, amount, names=["name"], result=None
):
    """


    Performs an iterative contribution Life Cycle Assessment (LCA) on the given datasets and LCIA methods and groups the results into a DataFrame.



    Parameters:



    datasets (list): A list of datasets to perform the LCA on.


    methods (list): A list of methods to use in the LCA.


    pct (float): The percentage limit for significant contributors.


    amount (float): The amount to use in the LCA.


    names (list, optional): A list of names to use in the LCA. Defaults to ['name'].


    result (list, optional): A list to store the results. If None, a new list will be created. Defaults to None.



    Returns:


    pd.DataFrame: A DataFrame containing the result of the LCA.



    """

    if result is None:
        result = []


    # Group contributions into 'other' using the specified percentage limit

    contribution_abs_result = group_into_others(
        contribution_LCA_to_df(datasets, methods, amount, names=names), pct
    )
    contribution_abs_result.index.names = ["dataset", "contributions"]


    # Append the first impact category results to the result list


    result.append(contribution_abs_result[impact_names[0]])


    # Calculate contribution percentages


    contribution_pct_result = contribution_abs_result / contribution_abs_result.sum()


    # Identify significant contributors
    significant_contributors = (
        contribution_pct_result[contribution_pct_result[impact_names[0]] > pct]
        .index.get_level_values(1)
        .to_list()
    )


    # Iterate over datasets to find and process significant contributors

    for dataset in datasets:


        techno_exc = [exc.input for exc in dataset.technosphere()]

        contr_datasets = [
            ds for ds in techno_exc if ds["name"] in significant_contributors
        ]


        for contr_ds in contr_datasets:


            try:

                # Find the amount multiplier for the current dataset

                amount_multiplier = next(
                    (
                        exc.amount
                        for exc in dataset.technosphere()
                        if exc["name"] == contr_ds["name"]
                    ),
                    0,
                )

                # Recursive call with updated amount


                iterative_contribution_LCA_to_df(
                    [contr_ds],
                    methods,
                    pct,
                    amount * amount_multiplier,
                    names=names,
                    result=result,
                )

            except Exception as e:

                print(f"An error occurred: {e}")
                pass


    # Concatenate results into a single DataFrame


    df = pd.DataFrame(pd.concat(result, axis=0))

    df = df[df.ne(0).any(axis=1)]
    return df

In [88]:
def LCA_to_df(datasets, methods):
    # Initialize an empty dictionary to store the results
    results = {}

    # Iterate over each method
    for method in methods:
        for impact_name in impact_names:
            # Switch the LCA method
            lca.switch_method(method)

            # Initialize an empty dictionary for this method
            method_results = {}

            # Iterate over each dataset
            for dataset in datasets:
                # Redo the LCA for this dataset
                lca.redo_lcia({dataset: 1})

                # Store the LCA score for this dataset
                method_results[dataset["name"]] = lca.score

            # Store the results for this method
            results[impact_name] = method_results

    # Convert the results dictionary to a DataFrame and return it
    return pd.DataFrame(results)

# Calculations

In [141]:
ds = [ds for ds in bw2data.Database("Poppies") if ds["name"] == "Poppies district"][0]
ds

'Poppies district' (unit, NL, None)

## District breakdown - climate change

In [232]:
chosen_methods = [
    ("IPCC 2021", "climate change", "global warming potential (GWP100)"),
    ("EF v3.0 EN15804", "climate change", "global warming potential (GWP100)"),
]
impact_names = ["IPCC 2021", "IPCC 2021 w bio C"]
lcia_methods = dict(zip(impact_names, chosen_methods))

In [233]:
district_cc_result = contribution_LCA_to_df(
    [ds],
    chosen_methods,
    amount=1,
    names=["name"],
).stack(future_stack=True, level=0)

In [234]:
district_cc_result

Poppies district  Embodied - building materials                                             IPCC 2021            1.908523e+06
                                                                                            IPCC 2021 w bio C   -1.595170e+06
                  End of life - building materials                                          IPCC 2021            7.736026e+04
                                                                                            IPCC 2021 w bio C    1.263605e+05
                  transport, freight, lorry >32 metric ton, EURO6                           IPCC 2021            3.330446e+04
                                                                                            IPCC 2021 w bio C    3.423418e+04
                  market for photovoltaic flat-roof installation, 3kWp, single-Si, on roof  IPCC 2021            3.878921e+05
                                                                                            IPCC 2021 w bio C    3.986

In [300]:
district_cc_result_reset = district_cc_result.loc[("Poppies district",)].reset_index()

In [301]:
district_cc_result_reset

,level_0,level_1,0
0,Embodied - building materials,IPCC 2021,1.908523e+06
1,Embodied - building materials,IPCC 2021 w bio C,-1.595170e+06
2,End of life - building materials,IPCC 2021,7.736026e+04
3,End of life - building materials,IPCC 2021 w bio C,1.263605e+05
4,"transport, freight, lorry >32 metric ton, EURO6",IPCC 2021,3.330446e+04
5,"transport, freight, lorry >32 metric ton, EURO6",IPCC 2021 w bio C,3.423418e+04
6,market for photovoltaic flat-roof installation...,IPCC 2021,3.878921e+05
7,market for photovoltaic flat-roof installation...,IPCC 2021 w bio C,3.986352e+05
8,"Heat supply infrastructure, District_ATES",IPCC 2021,2.074084e+05
9,"Heat supply infrastructure, District_ATES",IPCC 2021 w bio C,2.131706e+05


In [302]:
district_cc_result_reset.rename(
    columns={"level_1": "LCIA Methods", "level_0": "contributions", 0: "result"},
    inplace=True,
)

In [303]:
district_cc_result_reset

,contributions,LCIA Methods,result
0,Embodied - building materials,IPCC 2021,1.908523e+06
1,Embodied - building materials,IPCC 2021 w bio C,-1.595170e+06
2,End of life - building materials,IPCC 2021,7.736026e+04
3,End of life - building materials,IPCC 2021 w bio C,1.263605e+05
4,"transport, freight, lorry >32 metric ton, EURO6",IPCC 2021,3.330446e+04
5,"transport, freight, lorry >32 metric ton, EURO6",IPCC 2021 w bio C,3.423418e+04
6,market for photovoltaic flat-roof installation...,IPCC 2021,3.878921e+05
7,market for photovoltaic flat-roof installation...,IPCC 2021 w bio C,3.986352e+05
8,"Heat supply infrastructure, District_ATES",IPCC 2021,2.074084e+05
9,"Heat supply infrastructure, District_ATES",IPCC 2021 w bio C,2.131706e+05


In [304]:
district_cc_result_reset["contributions"] = district_cc_result_reset[
    "contributions"
].replace(
    {
        "Embodied - building materials": "Product stage - Buildings",
        "End of life - building materials": "End of life - Buildings",
        "Heat supply infrastructure, District_ATES": "Product stage - ATES",
        "market for electricity, low voltage": "Use - Grid electricity",
        "market for photovoltaic flat-roof installation, 3kWp, single-Si, on roof": "Product stage - Solar PV",
        "transport, freight, lorry >32 metric ton, EURO6": "Construction stage - Housing Module Transport",
    }
)

In [305]:
district_cc_result_reset

,contributions,LCIA Methods,result
0,Product stage - Buildings,IPCC 2021,1.908523e+06
1,Product stage - Buildings,IPCC 2021 w bio C,-1.595170e+06
2,End of life - Buildings,IPCC 2021,7.736026e+04
3,End of life - Buildings,IPCC 2021 w bio C,1.263605e+05
4,Construction stage - Housing Module Transport,IPCC 2021,3.330446e+04
5,Construction stage - Housing Module Transport,IPCC 2021 w bio C,3.423418e+04
6,Product stage - Solar PV,IPCC 2021,3.878921e+05
7,Product stage - Solar PV,IPCC 2021 w bio C,3.986352e+05
8,Product stage - ATES,IPCC 2021,2.074084e+05
9,Product stage - ATES,IPCC 2021 w bio C,2.131706e+05


In [306]:
district_cc_result_reset.pivot(
    index="LCIA Methods", columns="contributions", values="result"
)

contributions,Construction stage - Housing Module Transport,End of life - Buildings,Product stage - ATES,Product stage - Buildings,Product stage - Solar PV,Use - Grid electricity
LCIA Methods,,,,,,
IPCC 2021,33304.462213,77360.263206,207408.358287,1.908523e+06,387892.113905,437392.140385
IPCC 2021 w bio C,34234.180099,126360.527861,213170.586782,-1.595170e+06,398635.155653,448427.039432


In [307]:
district_cc_result_reset = district_cc_result_reset.pivot(
    index="LCIA Methods", columns="contributions", values="result"
)

In [308]:
new_order = [
    "Product stage - Buildings",
    "Product stage - Solar PV",
    "Product stage - ATES",
    "Construction stage - Housing Module Transport",
    "Use - Grid electricity",
    "End of life - Buildings",
]


district_cc_result_reset = district_cc_result_reset[new_order]

In [309]:
district_cc_result_reset

contributions,Product stage - Buildings,Product stage - Solar PV,Product stage - ATES,Construction stage - Housing Module Transport,Use - Grid electricity,End of life - Buildings
LCIA Methods,,,,,,
IPCC 2021,1.908523e+06,387892.113905,207408.358287,33304.462213,437392.140385,77360.263206
IPCC 2021 w bio C,-1.595170e+06,398635.155653,213170.586782,34234.180099,448427.039432,126360.527861


In [310]:
district_cc_result_reset.reset_index(inplace=True)

In [336]:
district_cc_result_reset.columns = [
    col.replace("contributions", "") for col in district_cc_result_reset.columns
]

In [339]:
district_cc_result_reset.set_index("LCIA Methods", inplace=True)

In [340]:
# Separate the data into positive and negative values
positive_data = district_cc_result_reset[district_cc_result_reset >= 0]
negative_data = district_cc_result_reset[district_cc_result_reset < 0]
positive_data = positive_data.apply(pd.to_numeric, errors="coerce").fillna(0)
negative_data = negative_data.apply(pd.to_numeric, errors="coerce").fillna(0)

color_discrete_map = {
    "Product stage - Buildings": "#FF4500",
    "Product stage - Solar PV": "#FFA500",
    "Product stage - ATES": "#FFD700",
    "Construction stage - Housing Module Transport": "#A9A9A9",
    "Use - Grid electricity": "#0000CD",
    "End of life - Buildings": "#A52A2A",
}

# Create a subplot with 2 rows
fig = sp.make_subplots(rows=2, cols=1, vertical_spacing=0)
# Add bar traces for each column in positive_data and negative_data
for col in positive_data.columns:
    fig.add_trace(
        go.Bar(
            x=positive_data.index,
            y=positive_data[col],
            name=col,
            marker_color=color_discrete_map[col],
        ),
        row=1,
        col=1,
    )
    fig.add_trace(
        go.Bar(
            x=negative_data.index,
            y=negative_data[col],
            name=col,
            marker_color=color_discrete_map[col],
            showlegend=False,
        ),
        row=2,
        col=1,
    )

# Calculate total values
district_cc_result_reset["total"] = district_cc_result_reset.sum(axis=1)

total = district_cc_result_reset.drop(columns="total").sum(axis=1)

# Add scatter traces for total values
fig.add_trace(
    go.Scatter(
        x=total.index,
        y=total.where(total > 0),
        mode="markers",
        name="Total",
        marker=dict(color="black", size=10),
    ),
    row=1,
    col=1,
)
fig.add_trace(
    go.Scatter(
        x=total.index,
        y=total.where(total < 0),
        mode="markers",
        name="Total",
        marker=dict(color="black", size=10),
        showlegend=False,
    ),
    row=2,
    col=1,
)

# Update layout to stack bars and adjust the width
fig.update_layout(
    barmode="stack",
    width=700,
    height=500,
    plot_bgcolor="white",
    bargap=0.4,
    annotations=[
        dict(
            x=-0.23,
            y=0.5,
            showarrow=False,
            text="Life cycle GHG emissions, kg CO2 eq",
            xref="paper",
            yref="paper",
            textangle=-90,
        )
    ],
    shapes=[
        dict(
            type="rect",
            xref="paper",
            yref="paper",
            x0=0.05,
            y0=0,
            x1=1,
            y1=1,
            line=dict(
                color="lightgray",
                width=1,
            ),
        )
    ],
    # title=""
)
# Hide x-axis tick labels for the first subplot
fig.update_xaxes(showticklabels=False, row=1, col=1)
# Make x-axis labels wrapped and horizontal for the second subplot
fig.update_xaxes(tickangle=0, tickfont=dict(size=11), row=2, col=1)

# Calculate the maximum absolute value across all your data
max_value = abs(total).max()

# Set the same range for y-axis in all subplots
fig.update_yaxes(range=[0, max_value + 5e5], dtick=1e6, row=1, col=1)
fig.update_yaxes(range=[-(max_value + 5e5), 0], dtick=1e6, row=2, col=1)

# Display the combined chart
plot(fig, filename="Poppies cc result - district breakdown.html")

'Poppies cc result - district breakdown.html'

In [341]:
district_cc_result_reset

,Product stage - Buildings,Product stage - Solar PV,Product stage - ATES,Construction stage - Housing Module Transport,Use - Grid electricity,End of life - Buildings,total
LCIA Methods,,,,,,,
IPCC 2021,1.908523e+06,387892.113905,207408.358287,33304.462213,437392.140385,77360.263206,3.051880e+06
IPCC 2021 w bio C,-1.595170e+06,398635.155653,213170.586782,34234.180099,448427.039432,126360.527861,-3.743421e+05


In [342]:
# Life cycle GHG emissions per m2 per year, assuming a lifetime of 50 years.
GFA = 5097.7
LT = 50
district_cc_result_reset["total"] / GFA / LT

LCIA Methods
IPCC 2021            11.973557
IPCC 2021 w bio C    -1.468670
Name: total, dtype: float64

In [466]:
chosen_methods = [("IPCC 2021", "climate change", "global warming potential (GWP100)")]
impact_names = ["IPCC 2021"]
lcia_methods = dict(zip(impact_names, chosen_methods))

In [467]:
cc_result = iterative_contribution_LCA_to_df(
    [ds], chosen_methods, pct=0.3, amount=1, names=["name"]
)
cc_result

IPCC 2021
dataset                                            contributions                                                   
Poppies district                                   Embodied - building materials                       1.908523e+06
                                                   End of life - building materials                    7.736026e+04
                                                   transport, freight, lorry >32 metric ton, EURO6     3.330446e+04
                                                   market for photovoltaic flat-roof installation,...  3.878921e+05
                                                   Heat supply infrastructure, District_ATES           2.074084e+05
                                                   market for electricity, low voltage                 4.373921e+05
Embodied - building materials                      Poppies district - construction phase - skeleton    1.321043e+06
                                                   Poppies district - construction phase - housing...  5.874793e+05
Poppies district - construction phase - skeleton   concrete, all types to generic market for concr...  5.736605e+05
                                                   reinforcing steel production                        5.757980e+05
                                                   market for sawnwood, board, softwood, raw, drie...  8.390969e+04
                                                   market for sawnwood, beam, softwood, dried (u=1...  8.708809e+04
                                                   stone wool production, packed                       1.860799e+02
                                                   limestone, crushed, washed to generic market fo...  4.010349e+02
concrete, all types to generic market for concr... market for concrete, 20MPa                          1.097499e+05
                                                   market for concrete, 25MPa                          2.999941e+05
                                                   market for concrete, 30MPa                          2.191956e+04
                                                   market for concrete, 35MPa                          1.419970e+05
reinforcing steel production                       hot rolling, steel                                  5.629990e+04
                                                   market for steel, low-alloyed                       2.040223e+05
                                                   market for steel, unalloyed                         3.154758e+05
Poppies district - construction phase - housing... oriented strand board production                    4.532748e+01
                                                   cement production, Portland                         2.970956e+04
                                                   gypsum fibreboard production                        1.067746e+04
                                                   gypsum plasterboard production                      1.669374e+04
                                                   stone wool production, packed                       2.367423e+04
                                                   glass fibre production                              8.598721e+03
                                                   cross-laminated timber production                   3.255872e+05
                                                   glued laminated timber production, average glue...  1.773438e+04
                                                   door production, inner, wood                        2.389666e+04
                                                   door production, outer, wood-glass                  1.308620e+05
cross-laminated timber production                  market for ammonium sulfate                         4.282410e+01
                                                   market for diesel, burned in building machine       1.625311e+04
                                              

In [468]:
cc_result.reset_index(inplace=True)

In [469]:
cc_result.head()

,dataset,contributions,IPCC 2021
0,Poppies district,Embodied - building materials,1.908523e+06
1,Poppies district,End of life - building materials,7.736026e+04
2,Poppies district,"transport, freight, lorry >32 metric ton, EURO6",3.330446e+04
3,Poppies district,market for photovoltaic flat-roof installation...,3.878921e+05
4,Poppies district,"Heat supply infrastructure, District_ATES",2.074084e+05


In [470]:
import plotly.graph_objects as go

# Step 1: Create a list of unique nodes
nodes = pd.concat([cc_result["dataset"], cc_result["contributions"]]).unique()

# Step 2: Create a mapping of nodes to unique integers
node_indices = {node: i for i, node in enumerate(nodes)}

# Step 3: Replace the 'dataset' and 'contributions' columns with their corresponding indices
cc_result["source"] = cc_result["dataset"].map(node_indices)
cc_result["target"] = cc_result["contributions"].map(node_indices)

# Step 4: Create a list of colors for the nodes
# colors = ['#a6cee3']*len(nodes)
colors = [
    "#1f77b4",
    "#ff7f0e",
    "#2ca02c",
    "#d62728",
    "#9467bd",
    "#8c564b",
    "#e377c2",
    "#7f7f7f",
    "#bcbd22",
    "#17becf",
]

# Step 5: Use the Plotly library to create a Sankey diagram
fig = go.Figure(
    data=[
        go.Sankey(
            node=dict(
                pad=15,
                thickness=20,
                line=dict(color="black", width=0.5),
                label=nodes,
                color=colors,
            ),
            link=dict(
                source=cc_result["source"],  # indices correspond to labels
                target=cc_result["target"],
                value=cc_result["IPCC 2021"],
            ),
        )
    ]
)

fig.update_layout(title_text="Sankey Diagram", font_size=14)
plot(fig, filename="Life cycle GHG emissions sankey.html")

'Life cycle GHG emissions sankey.html'

In [349]:
chosen_methods = [
    ("EF v3.0 EN15804", "climate change", "global warming potential (GWP100)")
]

impact_names = ["IPCC 2021 w bio C"]

lcia_methods = dict(zip(impact_names, chosen_methods))

In [350]:
cc_bio_result = iterative_contribution_LCA_to_df(
    [ds], chosen_methods, pct=0.2, amount=1, names=["name"]
)
cc_bio_result

IPCC 2021 w bio C
dataset                                            contributions                                                        
Poppies district                                   Embodied - building materials                           -1.595170e+06
                                                   End of life - building materials                         1.263605e+05
                                                   transport, freight, lorry >32 metric ton, EURO6          3.423418e+04
                                                   market for photovoltaic flat-roof installation,...       3.986352e+05
                                                   Heat supply infrastructure, District_ATES                2.131706e+05
                                                   market for electricity, low voltage                      4.484270e+05
Embodied - building materials                      Poppies district - construction phase - skeleton        -4.428267e+05
                                                   Poppies district - construction phase - housing...      -1.152343e+06
Poppies district - construction phase - skeleton   concrete, all types to generic market for concr...       5.862011e+05
                                                   reinforcing steel production                             6.012453e+05
                                                   market for sawnwood, board, softwood, raw, drie...      -8.769197e+05
                                                   market for sawnwood, beam, softwood, dried (u=1...      -7.539527e+05
                                                   stone wool production, packed                            1.858841e+02
                                                   limestone, crushed, washed to generic market fo...       4.134934e+02
market for sawnwood, board, softwood, raw, drie... board, softwood, raw, kiln drying to u=10%              -8.943697e+05
                                                   market for transport, freight train                      7.130539e+02
                                                   market for transport, freight, lorry, unspecified        1.673694e+04
market for sawnwood, beam, softwood, dried (u=1... planing, beam, softwood, u=10%                          -7.692353e+05
                                                   market for transport, freight train                      6.244862e+02
                                                   market for transport, freight, lorry, unspecified        1.465807e+04
Poppies district - construction phase - housing... oriented strand board production                        -1.717480e+02
                                                   cement production, Portland                              3.040566e+04
                                                   gypsum fibreboard production                             1.192182e+04
                                                   gypsum plasterboard production                           1.611031e+04
                                                   stone wool production, packed                            2.364933e+04
                                                   glass fibre production                                   8.887938e+03
                                                   cross-laminated timber production                       -1.266761e+06
                                                   glued laminated timber production, average glue...      -7.046314e+04
                                                   door production, inner, wood                            -1.157924e+04
                                                   door production, outer, wood-glass                       1.056570e+05
cross-laminated timber production                  market for ammonium sulfate                              4.372871e+01
                                                   market for diesel, burned in building machine            1.

In [351]:
cc_bio_result.reset_index(inplace=True)

In [352]:
cc_bio_result.head()

,dataset,contributions,IPCC 2021 w bio C
0,Poppies district,Embodied - building materials,-1.595170e+06
1,Poppies district,End of life - building materials,1.263605e+05
2,Poppies district,"transport, freight, lorry >32 metric ton, EURO6",3.423418e+04
3,Poppies district,market for photovoltaic flat-roof installation...,3.986352e+05
4,Poppies district,"Heat supply infrastructure, District_ATES",2.131706e+05


In [353]:
import plotly.graph_objects as go

# Step 1: Create a list of unique nodes
nodes = pd.concat([cc_bio_result["dataset"], cc_bio_result["contributions"]]).unique()

# Step 2: Create a mapping of nodes to unique integers
node_indices = {node: i for i, node in enumerate(nodes)}

# Step 3: Replace the 'dataset' and 'contributions' columns with their corresponding indices
cc_bio_result["source"] = cc_bio_result["dataset"].map(node_indices)
cc_bio_result["target"] = cc_bio_result["contributions"].map(node_indices)

# Step 4: Create a list of colors for the nodes
# colors = ['#a6cee3']*len(nodes)
colors = [
    "#1f77b4",
    "#ff7f0e",
    "#2ca02c",
    "#d62728",
    "#9467bd",
    "#8c564b",
    "#e377c2",
    "#7f7f7f",
    "#bcbd22",
    "#17becf",
]

# Step 5: Use the Plotly library to create a Sankey diagram
fig = go.Figure(
    data=[
        go.Sankey(
            node=dict(
                pad=15,
                thickness=20,
                line=dict(color="black", width=0.5),
                label=nodes,
                color=colors,
            ),
            link=dict(
                source=cc_bio_result["source"],  # indices correspond to labels
                target=cc_bio_result["target"],
                value=cc_bio_result["IPCC 2021 w bio C"],
            ),
        )
    ]
)

fig.update_layout(title_text="Sankey Diagram", font_size=10)
plot(fig, filename="Life cycle GHG emissions w bio C sankey.html")

'Life cycle GHG emissions w bio C sankey.html'

## District breakdown - cumulated non-renewable energy demand

In [602]:
chosen_methods = [
    (
        "Cumulative Energy Demand (CED)",
        "energy resources: non-renewable",
        "energy content (HHV)",
    )
]
impact_names = ["cumulative energy demand, non-renewable"]
lcia_methods = dict(zip(impact_names, chosen_methods))

In [380]:
district_ced_result = contribution_LCA_to_df(
    [ds],
    [chosen_methods],
    amount=1,
    names=["name"],
).stack(future_stack=True, level=0)

In [559]:
district_ced_result_reset = district_ced_result.loc[("Poppies district",)].reset_index()

In [560]:
district_ced_result_reset

,level_0,level_1,0
0,Embodied - building materials,"cumulative energy demand, non-renewable",2.302024e+07
1,End of life - building materials,"cumulative energy demand, non-renewable",1.141748e+06
2,"transport, freight, lorry >32 metric ton, EURO6","cumulative energy demand, non-renewable",5.410942e+05
3,market for photovoltaic flat-roof installation...,"cumulative energy demand, non-renewable",5.322178e+06
4,"Heat supply infrastructure, District_ATES","cumulative energy demand, non-renewable",2.180115e+06
5,"market for electricity, low voltage","cumulative energy demand, non-renewable",6.963641e+06


In [561]:
district_ced_result_reset.rename(
    columns={"level_1": "LCIA Methods", "level_0": "contributions", 0: "result"},
    inplace=True,
)

In [562]:
district_ced_result_reset

,contributions,LCIA Methods,result
0,Embodied - building materials,"cumulative energy demand, non-renewable",2.302024e+07
1,End of life - building materials,"cumulative energy demand, non-renewable",1.141748e+06
2,"transport, freight, lorry >32 metric ton, EURO6","cumulative energy demand, non-renewable",5.410942e+05
3,market for photovoltaic flat-roof installation...,"cumulative energy demand, non-renewable",5.322178e+06
4,"Heat supply infrastructure, District_ATES","cumulative energy demand, non-renewable",2.180115e+06
5,"market for electricity, low voltage","cumulative energy demand, non-renewable",6.963641e+06


In [563]:
district_ced_result_reset["contributions"] = district_ced_result_reset[
    "contributions"
].replace(
    {
        "Embodied - building materials": "Product stage - Buildings",
        "End of life - building materials": "End of life - Buildings",
        "Heat supply infrastructure, District_ATES": "Product stage - ATES",
        "market for electricity, low voltage": "Use - Grid electricity",
        "market for photovoltaic flat-roof installation, 3kWp, single-Si, on roof": "Product stage - Solar PV",
        "transport, freight, lorry >32 metric ton, EURO6": "Construction stage - Housing Module Transport",
    }
)

In [564]:
district_ced_result_reset

,contributions,LCIA Methods,result
0,Product stage - Buildings,"cumulative energy demand, non-renewable",2.302024e+07
1,End of life - Buildings,"cumulative energy demand, non-renewable",1.141748e+06
2,Construction stage - Housing Module Transport,"cumulative energy demand, non-renewable",5.410942e+05
3,Product stage - Solar PV,"cumulative energy demand, non-renewable",5.322178e+06
4,Product stage - ATES,"cumulative energy demand, non-renewable",2.180115e+06
5,Use - Grid electricity,"cumulative energy demand, non-renewable",6.963641e+06


In [565]:
district_ced_result_reset = district_ced_result_reset.pivot(
    index="LCIA Methods", columns="contributions", values="result"
)

In [566]:
new_order = [
    "Product stage - Buildings",
    "Product stage - Solar PV",
    "Product stage - ATES",
    "Construction stage - Housing Module Transport",
    "Use - Grid electricity",
    "End of life - Buildings",
]

district_ced_result_reset = district_ced_result_reset[new_order]

In [567]:
district_ced_result_reset

contributions,Product stage - Buildings,Product stage - Solar PV,Product stage - ATES,Construction stage - Housing Module Transport,Use - Grid electricity,End of life - Buildings
LCIA Methods,,,,,,
"cumulative energy demand, non-renewable",2.302024e+07,5.322178e+06,2.180115e+06,541094.176202,6.963641e+06,1.141748e+06


In [568]:
district_ced_result_reset.reset_index(inplace=True)

In [569]:
district_ced_result_reset

contributions,LCIA Methods,Product stage - Buildings,Product stage - Solar PV,Product stage - ATES,Construction stage - Housing Module Transport,Use - Grid electricity,End of life - Buildings
0,"cumulative energy demand, non-renewable",2.302024e+07,5.322178e+06,2.180115e+06,541094.176202,6.963641e+06,1.141748e+06


In [570]:
district_ced_result_reset.columns = [
    col.replace("contributions", "") for col in district_ced_result_reset.columns
]

In [571]:
district_ced_result_reset

,LCIA Methods,Product stage - Buildings,Product stage - Solar PV,Product stage - ATES,Construction stage - Housing Module Transport,Use - Grid electricity,End of life - Buildings
0,"cumulative energy demand, non-renewable",2.302024e+07,5.322178e+06,2.180115e+06,541094.176202,6.963641e+06,1.141748e+06


In [572]:
district_ced_result_reset.set_index("LCIA Methods", inplace=True)

In [573]:
district_ced_result_reset

,Product stage - Buildings,Product stage - Solar PV,Product stage - ATES,Construction stage - Housing Module Transport,Use - Grid electricity,End of life - Buildings
LCIA Methods,,,,,,
"cumulative energy demand, non-renewable",2.302024e+07,5.322178e+06,2.180115e+06,541094.176202,6.963641e+06,1.141748e+06


In [430]:
bw2data.methods[
    (
        "Cumulative Energy Demand (CED)",
        "energy resources: non-renewable",
        "energy content (HHV)",
    )
]["unit"]

'MJ-Eq'

In [438]:
district_ced_result_reset = district_ced_result_reset.loc[
    "cumulative energy demand, non-renewable"
]

In [451]:
district_ced_result_reset.loc["Product stage - Buildings"]

np.float64(23020244.753065262)

In [458]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Create a subplot
fig = make_subplots(rows=1, cols=1)

# Add bar traces for each column in district_ced_result_reset
for contributor in district_ced_result_reset.index:
    fig.add_trace(
        go.Bar(
            y=[district_ced_result_reset.loc[contributor]],
            name=contributor,
            marker_color=color_discrete_map[contributor],
        ),
        row=1,
        col=1,
    )

# Set the barmode to 'stack'
fig.update_layout(
    barmode="stack",
    width=500,
    height=500,
    plot_bgcolor="white",
    annotations=[
        dict(
            x=-1,
            y=0.5,
            showarrow=False,
            text="Life cycle cumulated energy demand, <br> non-renewable, MJ",
            xref="paper",
            yref="paper",
            textangle=-90,
        )
    ],
    shapes=[
        dict(
            type="rect",
            xref="paper",
            yref="paper",
            x0=0,
            y0=0,
            x1=1,
            y1=1,
            line=dict(
                color="lightgray",
                width=1,
            ),
        )
    ],
    # title=""
)
fig.update_xaxes(title_text="")
# Display the chart
plot(fig, filename="Poppies ced result - district breakdown.html")

'Poppies ced result - district breakdown.html'

In [604]:
ced_sankey_result = iterative_contribution_LCA_to_df(
    [ds], chosen_methods, pct=0.2, amount=1, names=["name"]
)
ced_sankey_result

cumulative energy demand, non-renewable
dataset                                            contributions                                                                              
Poppies district                                   Embodied - building materials                                                  2.302024e+07
                                                   End of life - building materials                                               1.141748e+06
                                                   transport, freight, lorry >32 metric ton, EURO6                                5.410942e+05
                                                   market for photovoltaic flat-roof installation,...                             5.322178e+06
                                                   Heat supply infrastructure, District_ATES                                      2.180115e+06
                                                   market for electricity, low voltage                                            6.963641e+06
Embodied - building materials                      Poppies district - construction phase - skeleton                               1.365727e+07
                                                   Poppies district - construction phase - housing...                             9.362973e+06
Poppies district - construction phase - skeleton   concrete, all types to generic market for concr...                             4.227089e+06
                                                   reinforcing steel production                                                   6.565605e+06
                                                   market for sawnwood, board, softwood, raw, drie...                             1.396281e+06
                                                   market for sawnwood, beam, softwood, dried (u=1...                             1.459802e+06
                                                   stone wool production, packed                                                  2.592378e+03
                                                   limestone, crushed, washed to generic market fo...                             5.904020e+03
concrete, all types to generic market for concr... market for concrete, 20MPa                                                     8.238809e+05
                                                   market for concrete, 25MPa                                                     2.246454e+06
                                                   market for concrete, 30MPa                                                     1.573875e+05
                                                   market for concrete, 35MPa                                                     9.993662e+05
reinforcing steel production                       hot rolling, steel                                                             9.155714e+05
                                                   market for steel, low-alloyed                                                  2.296379e+06
                                                   market for steel, unalloyed                                                    3.353654e+06
Poppies district - construction phase - housing... oriented strand board production                                               1.031021e+03
                                                   cement production, Portland                                                    1.229126e+05
                                                   gypsum fibreboard production                                                   1.699149e+05
                                                   gypsum plasterboard production                                                 2.635737e+05
                                                   stone wool production, packed                                                  3.298184e+05
                                                   glass fibre production                          

In [605]:
ced_sankey_result.reset_index(inplace=True)

In [606]:
ced_sankey_result.head()

,dataset,contributions,"cumulative energy demand, non-renewable"
0,Poppies district,Embodied - building materials,2.302024e+07
1,Poppies district,End of life - building materials,1.141748e+06
2,Poppies district,"transport, freight, lorry >32 metric ton, EURO6",5.410942e+05
3,Poppies district,market for photovoltaic flat-roof installation...,5.322178e+06
4,Poppies district,"Heat supply infrastructure, District_ATES",2.180115e+06


In [607]:
import plotly.graph_objects as go

# Step 1: Create a list of unique nodes
nodes = pd.concat(
    [ced_sankey_result["dataset"], ced_sankey_result["contributions"]]
).unique()

# Step 2: Create a mapping of nodes to unique integers
node_indices = {node: i for i, node in enumerate(nodes)}

# Step 3: Replace the 'dataset' and 'contributions' columns with their corresponding indices
ced_sankey_result["source"] = ced_sankey_result["dataset"].map(node_indices)
ced_sankey_result["target"] = ced_sankey_result["contributions"].map(node_indices)

# Step 4: Create a list of colors for the nodes
# colors = ['#a6cee3']*len(nodes)
colors = [
    "#1f77b4",
    "#ff7f0e",
    "#2ca02c",
    "#d62728",
    "#9467bd",
    "#8c564b",
    "#e377c2",
    "#7f7f7f",
    "#bcbd22",
    "#17becf",
]

# Step 5: Use the Plotly library to create a Sankey diagram
fig = go.Figure(
    data=[
        go.Sankey(
            node=dict(
                pad=15,
                thickness=20,
                line=dict(color="black", width=0.5),
                label=nodes,
                color=colors,
            ),
            link=dict(
                source=ced_sankey_result["source"],  # indices correspond to labels
                target=ced_sankey_result["target"],
                value=ced_sankey_result["cumulative energy demand, non-renewable"],
            ),
        )
    ]
)

fig.update_layout(title_text="Sankey Diagram", font_size=10)
plot(fig, filename="CED sankey.html")

'CED sankey.html'

## EF

In [482]:
chosen_methods = [
    ("EF v3.1", "acidification", "accumulated exceedance (AE)"),
    ("EF v3.1", "climate change", "global warming potential (GWP100)"),
    (
        "EF v3.1",
        "ecotoxicity: freshwater",
        "comparative toxic unit for ecosystems (CTUe)",
    ),
    (
        "EF v3.1",
        "energy resources: non-renewable",
        "abiotic depletion potential (ADP): fossil fuels",
    ),
    (
        "EF v3.1",
        "eutrophication: freshwater",
        "fraction of nutrients reaching freshwater end compartment (P)",
    ),
    (
        "EF v3.1",
        "eutrophication: marine",
        "fraction of nutrients reaching marine end compartment (N)",
    ),
    ("EF v3.1", "eutrophication: terrestrial", "accumulated exceedance (AE)"),
    (
        "EF v3.1",
        "human toxicity: carcinogenic",
        "comparative toxic unit for human (CTUh)",
    ),
    (
        "EF v3.1",
        "human toxicity: non-carcinogenic",
        "comparative toxic unit for human (CTUh)",
    ),
    (
        "EF v3.1",
        "ionising radiation: human health",
        "human exposure efficiency relative to u235",
    ),
    ("EF v3.1", "land use", "soil quality index"),
    (
        "EF v3.1",
        "material resources: metals/minerals",
        "abiotic depletion potential (ADP): elements (ultimate reserves)",
    ),
    ("EF v3.1", "ozone depletion", "ozone depletion potential (ODP)"),
    ("EF v3.1", "particulate matter formation", "impact on human health"),
    (
        "EF v3.1",
        "photochemical oxidant formation: human health",
        "tropospheric ozone concentration increase",
    ),
    (
        "EF v3.1",
        "water use",
        "user deprivation potential (deprivation-weighted water consumption)",
    ),
]
impact_names = [
    "Acidification",
    "Climate change",
    "Ecotoxicity, freshwater",
    "Resource depletion, fossils",
    "Eutrophication, freshwater",
    "Eutrophication, marine",
    "Eutrophication, terrestrial",
    "Human toxicity, cancer",
    "Human toxicity, non-cancer",
    "Ionising radiation",
    "Land use",
    "Resource depletion, minerals and metals",
    "Ozone depletion",
    "EF-particulate matter",
    "Photochemical ozone formation",
    "Water use",
]
lcia_methods = dict(zip(impact_names, chosen_methods))

In [483]:
district_ef_result = contribution_LCA_to_df(
    [ds],
    chosen_methods,
    amount=1,
    names=["name"],
).stack(future_stack=True, level=0)

In [484]:
district_ef_result

Poppies district  Embodied - building materials        Acidification                              9.890769e+03
                                                       Climate change                             1.908523e+06
                                                       EF-particulate matter                      2.803849e-01
                                                       Ecotoxicity, freshwater                    1.147144e+07
                                                       Eutrophication, freshwater                 6.620482e+02
                                                                                                      ...     
                  market for electricity, low voltage  Ozone depletion                            1.369925e-02
                                                       Photochemical ozone formation              7.629032e+02
                                                       Resource depletion, fossils                6.454833e+06
 

In [485]:
district_ef_result_reset = district_ef_result.loc[("Poppies district",)].reset_index()

In [486]:
district_ef_result_reset

,level_0,level_1,0
0,Embodied - building materials,Acidification,9.890769e+03
1,Embodied - building materials,Climate change,1.908523e+06
2,Embodied - building materials,EF-particulate matter,2.803849e-01
3,Embodied - building materials,"Ecotoxicity, freshwater",1.147144e+07
4,Embodied - building materials,"Eutrophication, freshwater",6.620482e+02
...,...,...,...
91,"market for electricity, low voltage",Ozone depletion,1.369925e-02
92,"market for electricity, low voltage",Photochemical ozone formation,7.629032e+02
93,"market for electricity, low voltage","Resource depletion, fossils",6.454833e+06
94,"market for electricity, low voltage","Resource depletion, minerals and metals",3.958589e+00


In [487]:
district_ef_result_reset.rename(
    columns={"level_1": "LCIA Methods", "level_0": "contributions", 0: "result"},
    inplace=True,
)

In [488]:
district_ef_result_reset

,contributions,LCIA Methods,result
0,Embodied - building materials,Acidification,9.890769e+03
1,Embodied - building materials,Climate change,1.908523e+06
2,Embodied - building materials,EF-particulate matter,2.803849e-01
3,Embodied - building materials,"Ecotoxicity, freshwater",1.147144e+07
4,Embodied - building materials,"Eutrophication, freshwater",6.620482e+02
...,...,...,...
91,"market for electricity, low voltage",Ozone depletion,1.369925e-02
92,"market for electricity, low voltage",Photochemical ozone formation,7.629032e+02
93,"market for electricity, low voltage","Resource depletion, fossils",6.454833e+06
94,"market for electricity, low voltage","Resource depletion, minerals and metals",3.958589e+00


In [489]:
district_ef_result_reset["contributions"] = district_ef_result_reset[
    "contributions"
].replace(
    {
        "Embodied - building materials": "Product stage - Buildings",
        "End of life - building materials": "End of life - Buildings",
        "Heat supply infrastructure, District_ATES": "Product stage - ATES",
        "market for electricity, low voltage": "Use - Grid electricity",
        "market for photovoltaic flat-roof installation, 3kWp, single-Si, on roof": "Product stage - Solar PV",
        "transport, freight, lorry >32 metric ton, EURO6": "Construction stage - Housing Module Transport",
    }
)

In [492]:
district_ef_result_reset = district_ef_result_reset.pivot(
    index="LCIA Methods", columns="contributions", values="result"
)

In [493]:
new_order = [
    "Product stage - Buildings",
    "Product stage - Solar PV",
    "Product stage - ATES",
    "Construction stage - Housing Module Transport",
    "Use - Grid electricity",
    "End of life - Buildings",
]

district_ef_result_reset = district_ef_result_reset[new_order]

In [494]:
district_ef_result_reset

contributions,Product stage - Buildings,Product stage - Solar PV,Product stage - ATES,Construction stage - Housing Module Transport,Use - Grid electricity,End of life - Buildings
LCIA Methods,,,,,,
Acidification,9.890769e+03,2.890006e+03,1.484271e+03,82.379302,1.008243e+03,1.518231e+03
Climate change,1.908523e+06,3.878921e+05,2.074084e+05,33304.462213,4.373921e+05,7.736026e+04
EF-particulate matter,2.803849e-01,2.673079e-02,1.351586e-02,0.003295,4.864292e-03,5.242044e-02
"Ecotoxicity, freshwater",1.147144e+07,3.550863e+06,3.076433e+06,241088.579408,9.333709e+05,6.177176e+05
"Eutrophication, freshwater",6.620482e+02,2.367158e+02,9.676942e+01,2.456110,1.655904e+02,5.175938e+00
"Eutrophication, marine",2.828012e+03,4.853758e+02,2.405121e+02,22.560149,2.461012e+02,3.789181e+02
"Eutrophication, terrestrial",3.116912e+04,4.626286e+03,2.503774e+03,230.422097,2.515827e+03,2.347596e+03
"Human toxicity, cancer",5.711939e-03,4.864321e-04,9.447806e-04,0.000016,1.666173e-04,2.432056e-04
"Human toxicity, non-cancer",2.320569e-02,2.099985e-02,1.265070e-02,0.000363,5.724727e-03,7.759362e-04


In [495]:
district_ef_result_reset.reset_index(inplace=True)

In [496]:
district_ef_result_reset

contributions,LCIA Methods,Product stage - Buildings,Product stage - Solar PV,Product stage - ATES,Construction stage - Housing Module Transport,Use - Grid electricity,End of life - Buildings
0,Acidification,9.890769e+03,2.890006e+03,1.484271e+03,82.379302,1.008243e+03,1.518231e+03
1,Climate change,1.908523e+06,3.878921e+05,2.074084e+05,33304.462213,4.373921e+05,7.736026e+04
2,EF-particulate matter,2.803849e-01,2.673079e-02,1.351586e-02,0.003295,4.864292e-03,5.242044e-02
3,"Ecotoxicity, freshwater",1.147144e+07,3.550863e+06,3.076433e+06,241088.579408,9.333709e+05,6.177176e+05
4,"Eutrophication, freshwater",6.620482e+02,2.367158e+02,9.676942e+01,2.456110,1.655904e+02,5.175938e+00
5,"Eutrophication, marine",2.828012e+03,4.853758e+02,2.405121e+02,22.560149,2.461012e+02,3.789181e+02
6,"Eutrophication, terrestrial",3.116912e+04,4.626286e+03,2.503774e+03,230.422097,2.515827e+03,2.347596e+03
7,"Human toxicity, cancer",5.711939e-03,4.864321e-04,9.447806e-04,0.000016,1.666173e-04,2.432056e-04
8,"Human toxicity, non-cancer",2.320569e-02,2.099985e-02,1.265070e-02,0.000363,5.724727e-03,7.759362e-04
9,Ionising radiation,1.594846e+05,3.502424e+04,8.163669e+03,637.127864,5.027542e+04,1.014796e+03


In [497]:
district_ef_result_reset.columns = [
    col.replace("contributions", "") for col in district_ef_result_reset.columns
]

In [498]:
district_ef_result_reset

,LCIA Methods,Product stage - Buildings,Product stage - Solar PV,Product stage - ATES,Construction stage - Housing Module Transport,Use - Grid electricity,End of life - Buildings
0,Acidification,9.890769e+03,2.890006e+03,1.484271e+03,82.379302,1.008243e+03,1.518231e+03
1,Climate change,1.908523e+06,3.878921e+05,2.074084e+05,33304.462213,4.373921e+05,7.736026e+04
2,EF-particulate matter,2.803849e-01,2.673079e-02,1.351586e-02,0.003295,4.864292e-03,5.242044e-02
3,"Ecotoxicity, freshwater",1.147144e+07,3.550863e+06,3.076433e+06,241088.579408,9.333709e+05,6.177176e+05
4,"Eutrophication, freshwater",6.620482e+02,2.367158e+02,9.676942e+01,2.456110,1.655904e+02,5.175938e+00
5,"Eutrophication, marine",2.828012e+03,4.853758e+02,2.405121e+02,22.560149,2.461012e+02,3.789181e+02
6,"Eutrophication, terrestrial",3.116912e+04,4.626286e+03,2.503774e+03,230.422097,2.515827e+03,2.347596e+03
7,"Human toxicity, cancer",5.711939e-03,4.864321e-04,9.447806e-04,0.000016,1.666173e-04,2.432056e-04
8,"Human toxicity, non-cancer",2.320569e-02,2.099985e-02,1.265070e-02,0.000363,5.724727e-03,7.759362e-04
9,Ionising radiation,1.594846e+05,3.502424e+04,8.163669e+03,637.127864,5.027542e+04,1.014796e+03


In [499]:
district_ef_result_reset.set_index("LCIA Methods", inplace=True)

In [500]:
district_ef_result_reset

,Product stage - Buildings,Product stage - Solar PV,Product stage - ATES,Construction stage - Housing Module Transport,Use - Grid electricity,End of life - Buildings
LCIA Methods,,,,,,
Acidification,9.890769e+03,2.890006e+03,1.484271e+03,82.379302,1.008243e+03,1.518231e+03
Climate change,1.908523e+06,3.878921e+05,2.074084e+05,33304.462213,4.373921e+05,7.736026e+04
EF-particulate matter,2.803849e-01,2.673079e-02,1.351586e-02,0.003295,4.864292e-03,5.242044e-02
"Ecotoxicity, freshwater",1.147144e+07,3.550863e+06,3.076433e+06,241088.579408,9.333709e+05,6.177176e+05
"Eutrophication, freshwater",6.620482e+02,2.367158e+02,9.676942e+01,2.456110,1.655904e+02,5.175938e+00
"Eutrophication, marine",2.828012e+03,4.853758e+02,2.405121e+02,22.560149,2.461012e+02,3.789181e+02
"Eutrophication, terrestrial",3.116912e+04,4.626286e+03,2.503774e+03,230.422097,2.515827e+03,2.347596e+03
"Human toxicity, cancer",5.711939e-03,4.864321e-04,9.447806e-04,0.000016,1.666173e-04,2.432056e-04
"Human toxicity, non-cancer",2.320569e-02,2.099985e-02,1.265070e-02,0.000363,5.724727e-03,7.759362e-04


In [501]:
new_order = [
    "Product stage - Buildings",
    "Product stage - Solar PV",
    "Product stage - ATES",
    "Construction stage - Housing Module Transport",
    "Use - Grid electricity",
    "End of life - Buildings",
]


district_ef_result_reset = district_ef_result_reset[new_order]

In [502]:
district_ef_result_reset

,Product stage - Buildings,Product stage - Solar PV,Product stage - ATES,Construction stage - Housing Module Transport,Use - Grid electricity,End of life - Buildings
LCIA Methods,,,,,,
Acidification,9.890769e+03,2.890006e+03,1.484271e+03,82.379302,1.008243e+03,1.518231e+03
Climate change,1.908523e+06,3.878921e+05,2.074084e+05,33304.462213,4.373921e+05,7.736026e+04
EF-particulate matter,2.803849e-01,2.673079e-02,1.351586e-02,0.003295,4.864292e-03,5.242044e-02
"Ecotoxicity, freshwater",1.147144e+07,3.550863e+06,3.076433e+06,241088.579408,9.333709e+05,6.177176e+05
"Eutrophication, freshwater",6.620482e+02,2.367158e+02,9.676942e+01,2.456110,1.655904e+02,5.175938e+00
"Eutrophication, marine",2.828012e+03,4.853758e+02,2.405121e+02,22.560149,2.461012e+02,3.789181e+02
"Eutrophication, terrestrial",3.116912e+04,4.626286e+03,2.503774e+03,230.422097,2.515827e+03,2.347596e+03
"Human toxicity, cancer",5.711939e-03,4.864321e-04,9.447806e-04,0.000016,1.666173e-04,2.432056e-04
"Human toxicity, non-cancer",2.320569e-02,2.099985e-02,1.265070e-02,0.000363,5.724727e-03,7.759362e-04


In [503]:
nf = pd.read_excel(
    r"K:\03_Staff\01_Current staff\Xiaojin Zhang\Data\Poppies\Normalisation_Weighting_Factors_EF_3.1.xlsx",
    sheet_name="NFs EF3.1",
    header=4,
    usecols="B:D",
    nrows=17,
)


nf["Impact categories"] = nf["Impact categories"].str.replace("*", "", regex=False)


nf.rename(columns={"Impact categories": "LCIA Methods"}, inplace=True)


nf.set_index("LCIA Methods", inplace=True)
nf

,Unit,NF
LCIA Methods,,
Acidification,mol H+ eq./person,55.569541
Climate change,kg CO2 eq./person,7553.083163
"Ecotoxicity, freshwater",CTUe/person,56716.586337
EF-particulate matter,disease incidences/person,0.000595
"Eutrophication, freshwater",kg P eq./person,1.606852
"Eutrophication, marine",kg N eq./person,19.545182
"Eutrophication, terrestrial",mol N eq./person,176.755000
"Human toxicity, cancer",CTUh/person,0.000017
"Human toxicity, non-cancer",CTUh/person,0.000129


In [504]:
wf = pd.read_excel(
    r"K:\03_Staff\01_Current staff\Xiaojin Zhang\Data\Poppies\Normalisation_Weighting_Factors_EF_3.1.xlsx",
    sheet_name="WFs",
    header=4,
    usecols="B:C",
    nrows=17,
)


wf.rename(columns={"Impact categories": "LCIA Methods"}, inplace=True)


wf.set_index("LCIA Methods", inplace=True)
wf

,WF [%]
LCIA Methods,
Acidification,0.0620
Climate change,0.2106
"Ecotoxicity, freshwater",0.0192
EF-particulate matter,0.0896
"Eutrophication, freshwater",0.0280
"Eutrophication, marine",0.0296
"Eutrophication, terrestrial",0.0371
"Human toxicity, cancer",0.0213
"Human toxicity, non-cancer",0.0184


In [505]:
n_district_ef_result = district_ef_result_reset.div(nf["NF"], axis=0)
n_district_ef_result

,Product stage - Buildings,Product stage - Solar PV,Product stage - ATES,Construction stage - Housing Module Transport,Use - Grid electricity,End of life - Buildings
LCIA Methods,,,,,,
Acidification,177.989038,52.007020,26.710150,1.482454,18.143808,27.321275
Climate change,252.681269,51.355467,27.460092,4.409386,57.909086,10.242210
EF-particulate matter,470.944732,44.898012,22.701740,5.534360,8.170244,88.047300
"Ecotoxicity, freshwater",202.258939,62.607132,54.242204,4.250760,16.456754,10.891305
"Eutrophication, freshwater",412.015645,147.316503,60.222977,1.528523,103.052668,3.221166
"Eutrophication, marine",144.691021,24.833527,12.305444,1.154256,12.591398,19.386777
"Eutrophication, terrestrial",176.340804,26.173440,14.165222,1.303624,14.233416,13.281636
"Human toxicity, cancer",331.071292,28.194226,54.760692,0.898465,9.657353,14.096506
"Human toxicity, non-cancer",180.258313,163.123706,98.268759,2.821040,44.468830,6.027356


In [506]:
w_district_ef_result = district_ef_result_reset.mul(wf["WF [%]"], axis=0)
w_district_ef_result

,Product stage - Buildings,Product stage - Solar PV,Product stage - ATES,Construction stage - Housing Module Transport,Use - Grid electricity,End of life - Buildings
LCIA Methods,,,,,,
Acidification,6.132277e+02,179.180386,92.024790,5.107517e+00,62.511073,94.130303
Climate change,4.019349e+05,81690.079188,43680.200255,7.013920e+03,92114.784765,16292.071431
EF-particulate matter,2.512248e-02,0.002395,0.001211,2.952297e-04,0.000436,0.004697
"Ecotoxicity, freshwater",2.202516e+05,68176.566362,59067.507108,4.628901e+03,17920.720924,11860.178188
"Eutrophication, freshwater",1.853735e+01,6.628043,2.709544,6.877109e-02,4.636531,0.144926
"Eutrophication, marine",8.370916e+01,14.367124,7.119159,6.677804e-01,7.284594,11.215975
"Eutrophication, terrestrial",1.156374e+03,171.635222,92.890010,8.548660e+00,93.337200,87.095796
"Human toxicity, cancer",1.216643e-04,0.000010,0.000020,3.301738e-07,0.000004,0.000005
"Human toxicity, non-cancer",4.269846e-04,0.000386,0.000233,6.682304e-06,0.000105,0.000014


In [507]:
w_district_ef_result.sum()

Product stage - Buildings                        3.789383e+07
Product stage - Solar PV                         7.237734e+05
Product stage - ATES                             3.248380e+05
Construction stage - Housing Module Transport    9.496426e+04
Use - Grid electricity                           7.652715e+05
End of life - Buildings                          2.476480e+05
dtype: float64

In [512]:
fig = go.Figure(
    data=[
        go.Bar(name=col, x=w_district_ef_result.index, y=w_district_ef_result[col])
        for col in w_district_ef_result.columns
    ]
)



# Change the bar mode



fig.update_layout(barmode="stack")

# Add Y-axis label
fig.update_yaxes(title_text="Environmetnal footprint points", type="log")


plot(fig, filename="Poppies total ef result - district breakdown.html")

'Poppies total ef result - district breakdown.html'

In [ ]:
fig.update_yaxes(type='log')

### Landuse

In [ ]:
ds_bm = [
    ds
    for ds in bw2data.Database("Poppies")
    if ds["name"] == "Embodied - building materials"
]

In [ ]:
ds_bm

['Embodied - building materials' (unit, NL, None)]

In [ ]:
chosen_methods = [("EF v3.1", "land use", "soil quality index")]
impact_names = ["Land use"]
lcia_methods = dict(zip(impact_names, chosen_methods))

In [510]:
bw2data.methods[("EF v3.1", "land use", "soil quality index")]["unit"]

'dimensionless'

In [ ]:
land_use_result = iterative_contribution_LCA_to_df(
    ds_bm, chosen_methods, pct=0.1, amount=1, names=["name"]
)
land_use_result

Land use
dataset                                            contributions                                                   
Embodied - building materials                      Poppies district - construction phase - skeleton    1.997256e+08
                                                   Poppies district - construction phase - housing...  2.461028e+08
Poppies district - construction phase - skeleton   concrete, all types to generic market for concr...  3.088172e+06
                                                   reinforcing steel production                        1.672620e+06
                                                   market for sawnwood, board, softwood, raw, drie...  1.016853e+08
                                                   market for sawnwood, beam, softwood, dried (u=1...  9.327424e+07
                                                   stone wool production, packed                       1.538190e+03
                                                   limestone, crushed, washed to generic market fo...  3.689754e+03
market for sawnwood, board, softwood, raw, drie... board, softwood, raw, kiln drying to u=10%          1.015023e+08
                                                   market for transport, freight train                 6.300280e+03
                                                   market for transport, freight, lorry, unspecified   1.766701e+05
market for sawnwood, beam, softwood, dried (u=1... planing, beam, softwood, u=10%                      9.311400e+07
                                                   market for transport, freight train                 5.517729e+03
                                                   market for transport, freight, lorry, unspecified   1.547261e+05
Poppies district - construction phase - housing... oriented strand board production                    2.073321e+04
                                                   cement production, Portland                         2.173535e+04
                                                   gypsum fibreboard production                        9.206488e+04
                                                   gypsum plasterboard production                      3.322010e+05
                                                   stone wool production, packed                       1.956981e+05
                                                   glass fibre production                              2.219440e+04
                                                   cross-laminated timber production                   2.245958e+08
                                                   glued laminated timber production, average glue...  1.441109e+07
                                                   door production, inner, wood                        2.688278e+06
                                                   door production, outer, wood-glass                  3.723020e+06
cross-laminated timber production                  market for ammonium sulfate                         2.069520e+02
                                                   market for diesel, burned in building machine       1.433539e+04
                                                   market group for electricity, medium voltage        2.596659e+05
                                                   market for furnace, wood chips, with silo, 300kW    2.167048e+04
                                                   market for lubricating oil                          4.226738e+03
                                                   market for melamine urea formaldehyde adhesive      6.881169e+04
                                                   market group for municipal solid waste              3.996490e+02
                                                   market for polyurethane adhesive                    7.448701e+04
                                                   market for sawlog and veneer log, softwood, mea...  6.513252e+07
                                               

In [ ]:
land_use_result.reset_index(inplace=True)

In [ ]:
land_use_result.head()

,dataset,contributions,Land use
0,Embodied - building materials,Poppies district - construction phase - skeleton,1.997256e+08
1,Embodied - building materials,Poppies district - construction phase - housin...,2.461028e+08
2,Poppies district - construction phase - skeleton,"concrete, all types to generic market for conc...",3.088172e+06
3,Poppies district - construction phase - skeleton,reinforcing steel production,1.672620e+06
4,Poppies district - construction phase - skeleton,"market for sawnwood, board, softwood, raw, dri...",1.016853e+08


In [ ]:
import plotly.graph_objects as go

# Step 1: Create a list of unique nodes
nodes = pd.concat(
    [land_use_result["dataset"], land_use_result["contributions"]]
).unique()

# Step 2: Create a mapping of nodes to unique integers
node_indices = {node: i for i, node in enumerate(nodes)}

# Step 3: Replace the 'dataset' and 'contributions' columns with their corresponding indices
land_use_result["source"] = land_use_result["dataset"].map(node_indices)
land_use_result["target"] = land_use_result["contributions"].map(node_indices)

# Step 4: Create a list of colors for the nodes
# colors = ['#a6cee3']*len(nodes)
colors = [
    "#1f77b4",
    "#ff7f0e",
    "#2ca02c",
    "#d62728",
    "#9467bd",
    "#8c564b",
    "#e377c2",
    "#7f7f7f",
    "#bcbd22",
    "#17becf",
]

# Step 5: Use the Plotly library to create a Sankey diagram
fig = go.Figure(
    data=[
        go.Sankey(
            node=dict(
                pad=15,
                thickness=20,
                line=dict(color="black", width=0.5),
                label=nodes,
                color=colors,
            ),
            link=dict(
                source=land_use_result["source"],  # indices correspond to labels
                target=land_use_result["target"],
                value=land_use_result["Land use"],
            ),
        )
    ]
)

fig.update_layout(title_text="Sankey Diagram", font_size=10)
plot(fig, filename="Land use sankey.html")

'Land use sankey.html'

### Resource depletion

In [ ]:
chosen_methods = [
    (
        "EF v3.1",
        "energy resources: non-renewable",
        "abiotic depletion potential (ADP): fossil fuels",
    )
]

impact_names = ["Resource depletion, fossils"]

lcia_methods = dict(zip(impact_names, chosen_methods))

In [511]:
bw2data.methods[
    (
        "EF v3.1",
        "energy resources: non-renewable",
        "abiotic depletion potential (ADP): fossil fuels",
    )
]["unit"]

'MJ, net calorific value'

In [ ]:
re_dep_result = iterative_contribution_LCA_to_df(
    ds_bm, chosen_methods, pct=0.2, amount=1, names=["name"]
)
re_dep_result

Resource depletion, fossils
dataset                                            contributions                                                                  
Embodied - building materials                      Poppies district - construction phase - skeleton                   1.285463e+07
                                                   Poppies district - construction phase - housing...                 8.829209e+06
Poppies district - construction phase - skeleton   concrete, all types to generic market for concr...                 3.969095e+06
                                                   reinforcing steel production                                       6.173660e+06
                                                   market for sawnwood, board, softwood, raw, drie...                 1.321198e+06
                                                   market for sawnwood, beam, softwood, dried (u=1...                 1.382706e+06
                                                   stone wool production, packed                                      2.416444e+03
                                                   limestone, crushed, washed to generic market fo...                 5.556608e+03
concrete, all types to generic market for concr... market for concrete, 20MPa                                         7.731419e+05
                                                   market for concrete, 25MPa                                         2.109819e+06
                                                   market for concrete, 30MPa                                         1.478148e+05
                                                   market for concrete, 35MPa                                         9.383187e+05
reinforcing steel production                       hot rolling, steel                                                 8.606918e+05
                                                   market for steel, low-alloyed                                      2.158055e+06
                                                   market for steel, unalloyed                                        3.154913e+06
Poppies district - construction phase - housing... oriented strand board production                                   9.680505e+02
                                                   cement production, Portland                                        1.162426e+05
                                                   gypsum fibreboard production                                       1.597446e+05
                                                   gypsum plasterboard production                                     2.466885e+05
                                                   stone wool production, packed                                      3.074349e+05
                                                   glass fibre production                                             1.343355e+05
                                                   cross-laminated timber production                                  5.685497e+06
                                                   glued laminated timber production, average glue...                 2.929327e+05
                                                   door production, inner, wood                                       3.626616e+05
                                                   door production, outer, wood-glass                                 1.522704e+06
cross-laminated timber production                  market for ammonium sulfate                                        6.349538e+02
                                                   market for diesel, burned in building machine                      2.143213e+05
                                                   market group for electricity, medium voltage                       1.754189e+06
                                                   market for furnace, wood chips, with silo, 300kW                   7.194345e+03
                                          

In [ ]:
re_dep_result.reset_index(inplace=True)

In [ ]:
re_dep_result.head()

,dataset,contributions,"Resource depletion, fossils"
0,Embodied - building materials,Poppies district - construction phase - skeleton,1.285463e+07
1,Embodied - building materials,Poppies district - construction phase - housin...,8.829209e+06
2,Poppies district - construction phase - skeleton,"concrete, all types to generic market for conc...",3.969095e+06
3,Poppies district - construction phase - skeleton,reinforcing steel production,6.173660e+06
4,Poppies district - construction phase - skeleton,"market for sawnwood, board, softwood, raw, dri...",1.321198e+06


In [ ]:
import plotly.graph_objects as go

# Step 1: Create a list of unique nodes
nodes = pd.concat([re_dep_result["dataset"], re_dep_result["contributions"]]).unique()

# Step 2: Create a mapping of nodes to unique integers
node_indices = {node: i for i, node in enumerate(nodes)}

# Step 3: Replace the 'dataset' and 'contributions' columns with their corresponding indices
re_dep_result["source"] = re_dep_result["dataset"].map(node_indices)
re_dep_result["target"] = re_dep_result["contributions"].map(node_indices)

# Step 4: Create a list of colors for the nodes
# colors = ['#a6cee3']*len(nodes)
colors = [
    "#1f77b4",
    "#ff7f0e",
    "#2ca02c",
    "#d62728",
    "#9467bd",
    "#8c564b",
    "#e377c2",
    "#7f7f7f",
    "#bcbd22",
    "#17becf",
]

# Step 5: Use the Plotly library to create a Sankey diagram
fig = go.Figure(
    data=[
        go.Sankey(
            node=dict(
                pad=15,
                thickness=20,
                line=dict(color="black", width=0.5),
                label=nodes,
                color=colors,
            ),
            link=dict(
                source=re_dep_result["source"],  # indices correspond to labels
                target=re_dep_result["target"],
                value=re_dep_result["Resource depletion, fossils"],
            ),
        )
    ]
)

fig.update_layout(title_text="Sankey Diagram", font_size=10)
plot(fig, filename="Resource depletion fossil sankey.html")

'Resource depletion fossil sankey.html'

## Considering future

In [513]:
em_sk = [
    ds
    for ds in bw2data.Database("Poppies")
    if ds["name"] == "Poppies district - construction phase - skeleton"
][0]


em_hm = [
    ds
    for ds in bw2data.Database("Poppies")
    if ds["name"] == "Poppies district - construction phase - housing modules"
][0]

In [557]:
all_result.groupby("location")["result"].sum()[
    all_result.groupby("location")["result"].sum().index == "NL"
]

location
NL    332990.841888
Name: result, dtype: float64

In [ ]:
calculation_setup = {
    "inv": [{ds: 1} for ds in elec_nl],
    "ia": list(lcia_methods.values()),
}

bw2data.calculation_setups["Dutch electricity supplies"] = calculation_setup

mlca = bw2calc.MultiLCA("Dutch electricity supplies")

mlca.results

array([[4.91316704e-01, 5.03712057e-01, 7.82216398e+00, 8.27242033e-10,
        8.98948713e-08, 3.93833379e-02]])

In [ ]:
# SSP most ambitious pathway
pd.DataFrame(mlca.results, columns=list(lcia_methods.values()), index=databases)

,"(IPCC 2021, climate change, global warming potential (GWP100))","(EF v3.0 EN15804, climate change, global warming potential (GWP100))","(Cumulative Energy Demand (CED), energy resources: non-renewable, energy content (HHV))","(ReCiPe 2016 v1.03, endpoint (I), total: ecosystem quality, ecosystem quality)","(ReCiPe 2016 v1.03, endpoint (I), total: human health, human health)","(ReCiPe 2016 v1.03, endpoint (I), total: natural resources, natural resources)"
ecoinvent 3.9.1 cutoff,0.491317,0.503712,7.822164,8.272420e-10,8.989487e-08,0.039383
ecoinvent_cutoff_3.9_remind_SSP2-PkBudg500_2030 2024-06-13,0.041196,0.035385,2.997666,2.699148e-10,3.149225e-08,0.005138
ecoinvent_cutoff_3.9_remind_SSP2-PkBudg500_2040 2024-06-13,0.020826,0.009437,1.201698,2.626031e-10,2.407306e-08,0.002276
ecoinvent_cutoff_3.9_remind_SSP2-PkBudg500_2050 2024-06-13,0.017605,0.007841,0.447612,2.347667e-10,1.995913e-08,0.001865
ecoinvent_cutoff_3.9_remind_SSP2-PkBudg500_2060 2024-06-13,0.016248,0.008580,0.225329,2.072669e-10,1.782120e-08,0.001738
ecoinvent_cutoff_3.9_remind_SSP2-PkBudg500_2070 2024-06-13,0.016065,0.011038,0.195346,1.929104e-10,1.673970e-08,0.001704
ecoinvent_cutoff_3.9_remind_SSP2-PkBudg500_2075 2024-06-13,0.016092,0.012590,0.192876,1.913469e-10,1.656315e-08,0.001694


## Geographical distribution

In [517]:
chosen_methods = [
    ("IPCC 2021", "climate change", "global warming potential (GWP100)"),
    ("EF v3.0 EN15804", "climate change", "global warming potential (GWP100)"),
]
impact_names = ["IPCC 2021", "IPCC 2021 w bio C"]
lcia_methods = dict(zip(impact_names, chosen_methods))

In [518]:
ds

'Poppies district' (unit, NL, None)

In [519]:
from constructive_geometries import *

geomatcher = Geomatcher()

In [520]:
region_mapping = pd.read_excel(
    "regionmapping.xlsx", sheet_name="regionmapping", keep_default_na=False
)

In [521]:
def replace_small_region_with_country(l):
    """
    this function will replace regions smaller than a country with the country it belongs
    it also deletes irelevant regions ecoinvent does not use
    """
    for idx, item in enumerate(l):
        if type(item) == tuple:
            if "Russia" in item[1]:
                l[idx] = "RU"
            else:
                l[idx] = item[1].split("-")[0]
    l = list(set([item for item in l if len(item) <= 3]))
    return l

In [522]:
# add a column "contained region", and include a list of countries that make up that region
# Initialize an empty list to store the items that caused a KeyError
skipped_items = []

for row in region_mapping.index:
    try:
        if pd.isna(region_mapping.loc[row, "Alpha-3 Code"]):
            region_mapping.loc[[row], "contained region"] = pd.Series(
                [
                    replace_small_region_with_country(
                        geomatcher.contained(
                            region_mapping.loc[row, "location"],
                            biggest_first=False,
                            exclusive=True,
                        )
                    )
                ],
                index=[row],
            )
    except KeyError:
        # If a KeyError occurs, append the current item to the skipped_items list
        skipped_items.append(row)

# Print the items that caused a KeyError
print("Skipped items:", skipped_items)

Europe,  not found in regex


Skipped items: [439]


In [523]:
region_mapping.loc[439]

Name                                                     Europe, without Russia and Turkey
location                                                 Europe, without Russia and Turkey
Geography Classification                                             ecoinvent Custom Area
Alpha-3 Code                                                                           NaN
Contained and Overlapping Geographies    AD; AL; AT; AX; BA; BALTSO; BE; BG; BV; BY; CE...
contained region                                                                       NaN
Name: 439, dtype: object

In [524]:
region_mapping[region_mapping["Name"] == "Europe"]["contained region"]

525    [PL, SJ, IM, GI, GR, HU, RU, AX, RO, FR, GG, B...
Name: contained region, dtype: object

In [525]:
list_of_countries = region_mapping[region_mapping["Name"] == "Europe"][
    "contained region"
].loc[525]

In [526]:
len(list_of_countries)

50

In [527]:
list_of_countries.remove("RU")
# list_of_countries.remove("TR")

In [528]:
len(list_of_countries)

49

In [529]:
region_mapping.loc[439]

Name                                                     Europe, without Russia and Turkey
location                                                 Europe, without Russia and Turkey
Geography Classification                                             ecoinvent Custom Area
Alpha-3 Code                                                                           NaN
Contained and Overlapping Geographies    AD; AL; AT; AX; BA; BALTSO; BE; BG; BV; BY; CE...
contained region                                                                       NaN
Name: 439, dtype: object

In [530]:
region_mapping.at[439, "contained region"] = list_of_countries

In [531]:
region_mapping.loc[439]

Name                                                     Europe, without Russia and Turkey
location                                                 Europe, without Russia and Turkey
Geography Classification                                             ecoinvent Custom Area
Alpha-3 Code                                                                           NaN
Contained and Overlapping Geographies    AD; AL; AT; AX; BA; BALTSO; BE; BG; BV; BY; CE...
contained region                         [PL, SJ, IM, GI, GR, HU, AX, RO, FR, GG, BA, M...
Name: 439, dtype: object

In [532]:
region_mapping = region_mapping.explode("contained region")

In [533]:
region_mapping[["location", "Alpha-3 Code"]].drop_duplicates()

,location,Alpha-3 Code
0,RME,NaN
1,TN,TUN
2,BG,BGR
3,UN-EASIA,NaN
4,GI,GIB
...,...,...
527,CF,CAF
528,NU,NIU
529,BR-RS,BRA
530,PY,PRY


In [534]:
# Convert two/three-digit locations to Alpha-3 Code for column "contained region"
region_mapping = region_mapping.merge(
    region_mapping[["location", "Alpha-3 Code"]].drop_duplicates(),
    left_on="contained region",
    right_on="location",
    how="left",
    suffixes=("", "_contained region"),
)

In [535]:
# Bring in GDP data
gdp_2015 = pd.read_csv("gdp2015.csv", skiprows=4, usecols=[0, 1, 2, 3, 59])

# correct alpha-3 code for Kosovo
gdp_2015.loc[261, "Country Code"] = "XXK"

region_mapping = region_mapping.merge(
    gdp_2015,
    left_on="Alpha-3 Code_contained region",
    right_on="Country Code",
    how="left",
)

In [580]:
# regional results weighted by the gdp of country
all_result = pd.DataFrame()
for dS in [ds]:
    lca = bw2calc.LCA({ds: 1}, chosen_methods[0])
    lca.lci()
    lca.lcia()
    rev_act_dict, rev_product_dict, rev_bio_dict = lca.reverse_dict()

    act_list = []
    for key in rev_act_dict.values():
        name = bw2data.get_activity(key).as_dict()["name"]
        location = bw2data.get_activity(key).as_dict()["location"]
        act_list.append([name, location])
    act_list = pd.DataFrame(act_list, columns=["name", "location"])

    ds_result = pd.concat(
        [act_list, pd.DataFrame(lca.characterized_inventory.sum(axis=0)).T], axis=1
    )
    ds_result.rename(columns={0: "result"}, inplace=True)
    ds_result["dataset name"] = ds["name"]
    ds_result = ds_result.groupby(["dataset name", "location"]).sum().reset_index()
    if ds_result.sum(axis=0)["result"] != 0:
        row_pct = (
            ds_result[ds_result["location"] == "RoW"].sum()["result"]
            / ds_result.sum(axis=0)["result"]
            * 100
        )
    else:
        row_pct = np.nan
    ds_result = ds_result.merge(region_mapping, on="location", how="left")
    ds_result = ds_result[ds_result["result"] != 0].copy()

    for row in ds_result.index:
        if (
            pd.isna(ds_result.loc[row, "Alpha-3 Code"])
            and ds_result.loc[row, "location"] != "RoW"
        ):
            total_result = ds_result[ds_result["Name"] == ds_result.loc[row, "Name"]][
                "result"
            ].sum()
            if total_result != 0:
                fr_of_gdp = ds_result.loc[row, "result"] / total_result
            else:
                fr_of_gdp = np.nan  # Handle divide by zero
            ds_result.loc[row, "gdp_w_result"] = (
                ds_result.loc[row, "result"] * fr_of_gdp
            )
        elif ds_result.loc[row, "location"] == "RoW":
            ds_result.loc[row, "gdp_w_result"] = ds_result.loc[row, "result"]
        else:
            ds_result.loc[row, "gdp_w_result"] = ds_result.loc[row, "result"]

    ds_result["Alpha-3 Code"].fillna(
        ds_result["Alpha-3 Code_contained region"], inplace=True
    )

    ds_result = (
        ds_result.groupby(["dataset name", "location", "Alpha-3 Code"], dropna=False)
        .sum()
        .reset_index()
    )

    all_result = pd.concat([all_result, ds_result], axis=0)

C:\Users\zhang_x\AppData\Local\Temp\2\ipykernel_13072\799695910.py:53: FutureWarning:

A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.





In [581]:
# Check the total life cycle GHG emissions for Poppies, and see if it equals to the value calculated previously.
all_result["gdp_w_result"].replace([np.inf, -np.inf], np.nan).sum(skipna=True)

np.float64(3049711.131003176)

In [554]:
import plotly.graph_objects as go
import numpy as np

# import chart_studio.plotly as py

fig = go.Figure(
    data=go.Choropleth(
        locations=all_result.groupby("Alpha-3 Code", dropna=False)
        .sum(numeric_only=True)
        .index,
        z=all_result.groupby("Alpha-3 Code", dropna=False).sum(numeric_only=True)[
            "result"
        ]
        / all_result.groupby("Alpha-3 Code", dropna=False)
        .sum(numeric_only=True)["result"]
        .sum()
        * 100,
        # colorscale=[
        # [0, "rgb(239, 243, 256)"],
        # [0.1, "rgb(49, 130, 189)"],
        # [0.1, "rgb(8, 81, 156)"],
        # ],
        colorscale="blues",
        marker_line_color="darkgray",
        marker_line_width=0.5,
        colorbar_title="% of total GHG " + "<br>emissions per year",
        hovertemplate="%{z:.1f}%",
        colorbar_ticksuffix="%",
        zmax=1,
        zmin=0,
    )
)

RoW_pct = int(
    all_result.groupby("Alpha-3 Code", dropna=False)
    .sum(numeric_only=True)
    .loc[np.nan]["result"]
    / all_result.groupby("Alpha-3 Code", dropna=False)
    .sum(numeric_only=True)["result"]
    .sum()
    * 100
)
GLO_pct = int(
    all_result.groupby("location", dropna=False)
    .sum(numeric_only=True)
    .loc["GLO"]["result"]
    / all_result.groupby("location", dropna=False)
    .sum(numeric_only=True)["result"]
    .sum()
    * 100
)

fig.update_layout(
    title_text="Global distribution of life cycle GHG Emissions caused by Poppies"
    + "<br>Contribution from the rest of the world: "
    + str(RoW_pct)
    + "% (not shown on map)"
    + "<br>Contribution from the globe: "
    + str(GLO_pct)
    + "%",
    geo=dict(showframe=False, showcoastlines=False, projection_type="equirectangular"),
    width=1000,
    height=600,
)
# ],
# )

fig.write_html("map.html")